In [3]:
import pandas as pd


In [4]:
pip install scikit-surprise


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from surprise import Dataset, Reader, KNNBasic

In [6]:
from surprise.model_selection import cross_validate

In [7]:
ratings = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')

Read for more info- https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [8]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
data = pd.merge(ratings,movies, on='movieId')

In [10]:
data.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [11]:
#scale of ratings

reader = Reader(rating_scale=(0.5,5))

In [12]:
data = Dataset.load_from_df(data[['userId', 'movieId','rating']], reader)

In [13]:
data

In [14]:
# set parameters for knn basic model- type of similarity and if we are doing user based/ item based filtering by putting true/ false 
# (item based)

params = {'name':'cosine', 'user_based':False}

In [15]:
model = KNNBasic(sim_options=params)

In [16]:
# doing five splits of data

cross_validate(model, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9756  0.9820  0.9725  0.9732  0.9721  0.9751  0.0037  
MAE (testset)     0.7569  0.7632  0.7568  0.7556  0.7554  0.7576  0.0029  
Fit time          4.97    3.94    4.23    4.20    3.34    4.13    0.53    
Test time         2.51    2.42    2.38    2.52    2.29    2.42    0.09    


{'test_rmse': array([0.97560156, 0.98198909, 0.9724664 , 0.97318865, 0.97208018]),
 'test_mae': array([0.75690372, 0.76317311, 0.7567502 , 0.75564223, 0.75537526]),
 'fit_time': (4.9677958488464355,
  3.937112808227539,
  4.228115797042847,
  4.203036069869995,
  3.3351097106933594),
 'test_time': (2.5093348026275635,
  2.4172332286834717,
  2.3801259994506836,
  2.520491600036621,
  2.290130138397217)}

we're trying to see how accurate the predictions are, can look at RMSE which is pretty accurate (0.97) and every fold is sort of the same so we are one mark off the actual rating, test time will tell how efficient the model is

In [17]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [18]:
## split the data into training and testing and determine the test size

trainset, testset = train_test_split(data, test_size=0.2)


In [21]:
model2 = KNNBasic(min_k = 5, k=10, sim_options={'name':'cosine','user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9935


0.9935249291195132

In [22]:
model2 = KNNBasic(min_k = 5, k=10, sim_options={'name':'msd','user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9471


0.947063121298483

In [23]:
model2.predict(668,16)

Prediction(uid=668, iid=16, r_ui=None, est=3.543960431502433, details={'actual_k': 10, 'was_impossible': False})

In [26]:
movies[movies['movieId']==16]

,movieId,title,genres
15,16,Casino (1995),Crime|Drama


In [27]:
final_dataset = pd.merge(movies, ratings, on='movieId')

In [28]:
final_dataset['predicted_rating'] = final_dataset.apply(lambda x:model2.predict(x['userId'],x['movieId']).est, axis=1)

In [29]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,4.955749
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,4.000000
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,3.565463
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.725120
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.000000


In [30]:
#SVD Algo will have epochs for optimization where default setting is 20

from surprise import SVD

In [31]:
svd= SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8747  0.8664  0.8670  0.8781  0.8650  0.8702  0.0052  
Fit time          0.55    0.46    0.45    0.42    0.42    0.46    0.05    
Test time         0.04    0.04    0.04    0.04    0.04    0.04    0.00    


{'test_rmse': array([0.87468973, 0.866442  , 0.86696333, 0.87810472, 0.86502925]),
 'fit_time': (0.5520541667938232,
  0.45557308197021484,
  0.44545793533325195,
  0.4165823459625244,
  0.41684913635253906),
 'test_time': (0.039010047912597656,
  0.03903388977050781,
  0.03601408004760742,
  0.03539586067199707,
  0.035784006118774414)}

predictions are a little bit better than the previous model, look at the average RMSE, this algo is much faster than KNNBasics 

In [32]:
trainset = data.build_full_trainset()

In [33]:
svd.fit(trainset)

In [34]:
final_dataset['svd_pred'] = final_dataset.apply(lambda x: svd.predict(x['userId'],x['movieId']).est, axis=1)

In [35]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating,svd_pred
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,4.955749,4.372071
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,4.000000,3.345603
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,3.565463,4.411028
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.725120,3.776781
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.000000,3.793625


Practice Question for Class-

pick any other regression based model, use pandas dataframe, try training the other pred model for the rating and see how regular model compares to the other models